In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

# Import Dataset

In [2]:
# Importando o dataset com as variáveis do modelo churn 
churn = pd.read_parquet('../data/raw/3m/churn.parquet')

# Importando o dataset com as variáveis do modelo p-alive
palive = pd.read_parquet('../data/raw/3m/palive.parquet')

# Importando o dataset com as variáveis de costumers e nps (feito pelo Francisco notebook1)
nps = pd.read_parquet('../data/stage/3m/customers_nps_ig.parquet')

In [3]:
churn['p_churn'] = churn['p_churn'].fillna(0)
churn = churn.drop_duplicates(subset='consumer_id')

In [4]:
#2021-11-12
#145926 - 1737544 - 709969
startdate = pd.to_datetime("2021-10-24").date()
enddate = pd.to_datetime("2021-10-31").date()
palive = palive[(palive['base_date'] > startdate) & (palive['base_date'] <= enddate)]
#palive = palive[palive['base_date'] > startdate]

In [5]:
nps['nps']  = nps['nps'] .map({0:0,
                                   1:0,
                                   2:0,
                                   3:0,
                                   4:0,
                                   5:0,
                                   6:0,
                                   7:0,
                                   8:1,
                                   9:1,
                                   10:1
                             },
                             na_action=None)

In [6]:
nps['nps'].value_counts()

1    3265
0    1797
Name: nps, dtype: int64

In [7]:
palive['consumer_id'].value_counts()

1715616    1
6796461    1
7425925    1
3969927    1
7282242    1
          ..
6613504    1
5553972    1
245032     1
6325798    1
1342034    1
Name: consumer_id, Length: 727974, dtype: int64

In [8]:
nps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5062 entries, 20663 to 9048056
Data columns (total 59 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   consumer_id                     5062 non-null   int64         
 1   role_id                         5062 non-null   int64         
 2   cpf                             5062 non-null   float64       
 3   first_name                      5062 non-null   object        
 4   last_name                       5062 non-null   object        
 5   email                           5062 non-null   object        
 6   birthdate                       5062 non-null   object        
 7   zipcode                         4356 non-null   float64       
 8   street                          4517 non-null   object        
 9   street_number                   4297 non-null   float64       
 10  complement                      4443 non-null   object        
 1

In [9]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4101417 entries, 0 to 4166750
Data columns (total 7 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   consumer_id               int32  
 1   ref_date                  object 
 2   p_churn                   float64
 3   status_criticidade_churn  object 
 4   categoria_motivo          object 
 5   motivo                    object 
 6   run_id                    object 
dtypes: float64(1), int32(1), object(5)
memory usage: 234.7+ MB


In [10]:
palive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 727974 entries, 731162 to 2187376
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   consumer_id         727974 non-null  int64  
 1   frequency           727974 non-null  float64
 2   recency             727974 non-null  float64
 3   T                   727974 non-null  int64  
 4   p_currently_alive   727974 non-null  float64
 5   p_alive_mean        699640 non-null  float64
 6   p_alive_ratio       699640 non-null  float64
 7   status_criticidade  727974 non-null  object 
 8   processing_date     727974 non-null  object 
 9   base_date           727974 non-null  object 
dtypes: float64(5), int64(2), object(3)
memory usage: 61.1+ MB


# Merge 'nps' + 'churn'

In [11]:
# merge entre 'nps' e 'churn'
nps_churn = pd.merge(nps, churn, on='consumer_id', how='left')

In [12]:
nps_churn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5062 entries, 0 to 5061
Data columns (total 65 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   consumer_id                     5062 non-null   int64         
 1   role_id                         5062 non-null   int64         
 2   cpf                             5062 non-null   float64       
 3   first_name                      5062 non-null   object        
 4   last_name                       5062 non-null   object        
 5   email                           5062 non-null   object        
 6   birthdate                       5062 non-null   object        
 7   zipcode                         4356 non-null   float64       
 8   street                          4517 non-null   object        
 9   street_number                   4297 non-null   float64       
 10  complement                      4443 non-null   object        
 11  city

# Merge entre 'nps_churn' + 'palive'

In [13]:
# merge entre 'nps' e 'palive'
nps_palive = pd.merge(nps_churn, palive, on='consumer_id', how='left')

In [14]:
nps_palive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5062 entries, 0 to 5061
Data columns (total 74 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   consumer_id                     5062 non-null   int64         
 1   role_id                         5062 non-null   int64         
 2   cpf                             5062 non-null   float64       
 3   first_name                      5062 non-null   object        
 4   last_name                       5062 non-null   object        
 5   email                           5062 non-null   object        
 6   birthdate                       5062 non-null   object        
 7   zipcode                         4356 non-null   float64       
 8   street                          4517 non-null   object        
 9   street_number                   4297 non-null   float64       
 10  complement                      4443 non-null   object        
 11  city

In [16]:
columns = ['consumer_id', 'role_id', 'cpf', 'email',
       'birthdate', 'city', 'state', 'phonenumber', 'created_at', 'updated_at',
       'eou_synced_at', 'last_activity_date', 'completed_at',
       'paypal_linked_at', 'braintree_id', 'braintree_token', 'paypal_id',
       'paypal_billing_agreement', 'paypal_email', 'paypal_cpf',
       'paydiant_uri', 'paydiant_linked_at', 'stone_wallet_id',
       'stone_linked_at', 'allow_newsletter', 'allow_promos',
       'paypal_allow_automatic_payment', 'allow_automatic_payment',
       'paypal_subscription_canceled', 'active', 'signup_complete',
       'legacy_email', 'legacy_paypal_id', 'legacy_paypal_subscription',
       'city_id', 'referral_code', 'mercadopago_linked_at',
       'mercadopago_email', 'mercadopago_id', 'mfa_validated', 'country',
       'nota_recomendacao_customer', 'nota_recomendacao_nps', 'nps',
       'p_churn', 'p_currently_alive']

In [18]:
nps_palive = nps_palive[columns]

# Salvando

In [22]:
#Salvando em Parquet
nps_palive.to_parquet('../data/stage/3m/customers_nps_churn_palive.parquet')